sheet flow:

- ~~define track of interest based on sheet_gid dict~~
- ~~read in weekend dataframe~~
- ~~run function drop_empties~~
- ~~run function check_df~~
- ~~produce these lists:~~
    - ~~predicted_qualifying_order, actual_qualifying_order, predicted_race_order, actual_race_order~~
- ~~produce these dfs:~~
    - ~~constructor_pricing, driver_pricing~~
- ~~produce these lists:~~
    - ~~drivers, constructors~~
- ~~set up constants for tracking weekend scores~~
    - ~~constructor_scores, driver_scores~~
- ~~run function score_qualification_order~~
- run function score_race_order
- run function score_fastest_pitstop
- run function score_fastest_lap
    
still to-do:
- ~~changing the overtake bonus/gain/loss points from estimations to direct reports in fantasy~~
- ~~produce df of driver and constructor pricing for making team affordability calculations each week, later when i get to the full program part of it~~
- ~~incorporate the team recommendation function that the post i found used~~
- ~~write function scoring the predictions only~~
    - ~~assume difference between predicted qualifying position and predicted race position are direct gain/loss~~
- ~~incorporate the pieces into scoring all of the race into one function~~
    - ~~qualifying, race, superlatives, overtakes+gain/loss~~
- make google sheet of teams and track team composition through each weekend
    - read this in, figure out if i can define a range of cells so i can do all three teams on the same google sheet or if i'm going to have to do a separate sheet for each
- ~~add scoring for my team to use for the team recommendations~~
- ~~add ability to handle DNF, DNQ, and OUT (sainz, stroll, zhou at saudi arabia)~~

In [1]:
from weekend_functions import *

### Define Track and Read In Weekend Data

In [2]:
track_name = 'australia'

In [9]:
# read in from google sheets the sheet for this weekend's events
weekend_df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid={sheet_gid[track_name]}')

# read in from local sometimes needed when offline, make sure to download the file from sheets first...
# weekend_df = pd.read_csv(f'f1_main - {track_name}.csv')

weekend_df = drop_empties(weekend_df)
check_df(weekend_df)

weekend_df

Column "fp2_australia" has an issue with the finishing positins represented in the data!
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 400]


Column "predicted_qualifying_australia" has an issue with the finishing positins represented in the data!
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, nan]


Column "predicted_race_australia" has an issue with the finishing positins represented in the data!
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, nan]




,Team,Driver,fp1_australia,fp2_australia,predicted_qualifying_australia,predicted_race_australia
0,Red Bull Racing-RBPT,Max Verstappen,2,2,1.0,1.0
1,Ferrari,Charles Leclerc,4,1,2.0,2.0
2,Red Bull Racing-RBPT,Sergio Perez,6,8,5.0,3.0
3,McLaren-Mercedes,Lando Norris,1,9,3.0,4.0
4,McLaren-Mercedes,Oscar Piastri,10,7,7.0,5.0
5,Ferrari,Carlos Sainz Jr.,8,3,4.0,6.0
6,Mercedes,Lewis Hamilton,9,18,10.0,7.0
7,Mercedes,George Russell,3,6,9.0,8.0
8,Aston Martin Aramco-Mercedes,Fernando Alonso,18,5,6.0,9.0
9,Aston Martin Aramco-Mercedes,Lance Stroll,7,4,8.0,10.0


In [10]:
weekend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Team                            20 non-null     object 
 1   Driver                          20 non-null     object 
 2   fp1_australia                   20 non-null     int64  
 3   fp2_australia                   20 non-null     int64  
 4   predicted_qualifying_australia  19 non-null     float64
 5   predicted_race_australia        19 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.1+ KB


### Define Predicted Qualifying, Actual Qualifying, Predicted Race, Actual Race Orders, Driver Gain/Loss

this was used when writing functions to score the actual race. its not needed for what this sheet has become, picking a team for the next race. i'll move it to another sheet eventually when if/when i need it still. or i'll delete it.

In [5]:
# # qualifying orders
# predicted_qualifying_order = weekend_df[['Driver', f'predicted_qualifying_{track_name}']].sort_values(by=[f'predicted_qualifying_{track_name}'])['Driver'].tolist()
# actual_qualifying_order = weekend_df[['Driver', f'actual_qualifying_{track_name}']].sort_values(by=[f'actual_qualifying_{track_name}'])['Driver'].tolist()

# # race orders
# predicted_race_order = weekend_df[['Driver', f'predicted_race_{track_name}']].sort_values(by=[f'predicted_race_{track_name}'])['Driver'].tolist()
# if f'actual_race_{track_name}' in weekend_df.columns:
#     actual_race_order = weekend_df[['Driver', f'actual_race_{track_name}']].sort_values(by=[f'actual_race_{track_name}'])['Driver'].tolist()

# # driver_gain_loss
# driver_gain_loss_overtake = {}
# for driver in weekend_df.Driver:
#     driver_gain_loss_overtake[driver] = {}
#     driver_gain_loss_overtake[driver]['gain_loss'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'positions_gained_{track_name}']].values[0][0]
#     driver_gain_loss_overtake[driver]['overtake'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'overtake_bonus_{track_name}']].values[0][0]

### Driver and Constructor Prices

In [11]:
# pricing for drivers and constructors, updated each week with the latest values from the Formula 1 Fantasy Game
driver_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=920234107').dropna(axis=1, how='all')
constructor_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=1972297135').dropna(axis=1, how='all')

### Look at Predictions for the Weekend

In [14]:
drivers, constructors, driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = score_race_qualifying_predicted(weekend_df, track_name)

In [18]:
driver_scores

{'Max Verstappen': 45.0,
 'Charles Leclerc': 27.0,
 'Sergio Perez': 25.0,
 'Lando Norris': 19.0,
 'Oscar Piastri': 18.0,
 'Carlos Sainz Jr.': 13.0,
 'Lewis Hamilton': 13.0,
 'George Russell': 8.0,
 'Fernando Alonso': 4.0,
 'Lance Stroll': 2.0,
 'Yuki Tsunoda': 4.0,
 'Daniel Ricciardo': -1.0,
 'Alexander Albon': -1.0,
 'Kevin Magnussen': 0.0,
 'Nico Hulkenberg': 0.0,
 'Valtteri Bottas': 4.0,
 'Pierre Gasly': 4.0,
 'Guanyu Zhou': -2.0,
 'Esteban Ocon': -2.0}

In [19]:
constructor_scores

{'Red Bull Racing-RBPT': 70.0,
 'Ferrari': 50.0,
 'McLaren-Mercedes': 47.0,
 'Mercedes': 31.0,
 'Aston Martin Aramco-Mercedes': 16.0,
 'VisaCashApp RB': 6.0,
 'Williams-Mercedes': 0.0,
 'Haas-Ferrari': 3.0,
 'Stake F1 Team Kick Sauber': 1.0,
 'Alpine-Renault': 1.0}

# Evaulate Next Race

In [12]:
# F-winning
my_constructors = ['Ferrari', 'VisaCashApp RB']
my_drivers = ['Daniel Ricciardo', 'Carlos Sainz Jr.', 'Fernando Alonso', 'Oscar Piastri', 'Nico Hulkenberg']
remaining_cost_cap = 3.5

# # F-winning-maths
# my_constructors = ['Ferrari', 'VisaCashApp RB']
# my_drivers = ['Max Verstappen', 'Daniel Ricciardo', 'Charles Leclerc', 'Alexander Albon', 'Nico Hulkenberg']
# remaining_cost_cap = 0.7

In [13]:
top_teams = main(my_drivers, my_constructors, weekend_df, track_name, driver_pricing, constructor_pricing, remaining_cost_cap)

=== Predicted Driver Scores for Australia ===
{'Max Verstappen': 45.0, 'Charles Leclerc': 27.0, 'Sergio Perez': 25.0, 'Lando Norris': 19.0, 'Oscar Piastri': 18.0, 'Carlos Sainz Jr.': 13.0, 'Lewis Hamilton': 13.0, 'George Russell': 8.0, 'Fernando Alonso': 4.0, 'Lance Stroll': 2.0, 'Yuki Tsunoda': 4.0, 'Daniel Ricciardo': -1.0, 'Alexander Albon': -1.0, 'Kevin Magnussen': 0.0, 'Nico Hulkenberg': 0.0, 'Valtteri Bottas': 4.0, 'Pierre Gasly': 4.0, 'Guanyu Zhou': -2.0, 'Esteban Ocon': -2.0}

=== Current Team ===
Constructors: ['Ferrari', 'VisaCashApp RB']
Drivers: ['Daniel Ricciardo', 'Carlos Sainz Jr.', 'Fernando Alonso', 'Oscar Piastri', 'Nico Hulkenberg']
Current Team Value: 101.5
Current Available Value: 3.5
Total Number of Team Combinations: 523260
Total Number of Team Combinations I can afford: 299341
Explored all of the valid 299341 teams.

=== TEAM AT POSITION 1 WITH SCORE 167.0 ===
Constructor: ('Ferrari', 'VisaCashApp RB')
Drivers: ('Max Verstappen', 'Oscar Piastri', 'Daniel Ricciar